In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("MAT")
Pkg.add("Printf")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("JLD")
Pkg.add("Plots")

using JuMP
using Gurobi
using MAT
using Printf
using DataFrames
using CSV
using Plots
using JLD

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Ch

In [28]:
# High-level Settings
Zone = "WEST" # price zone name

# read RTP price
fileln = matopen(string("./RTP_",Zone,"_2019_2021_1h.mat"))
RTP = read(fileln, string("RTP_",Zone,"_2019_2021_1h"))
close(fileln)

# read DAP price
fileln = matopen(string("./DAP_",Zone,"_2019_2021.mat"))
DAP = read(fileln, string("DAP_",Zone,"_2019_2021_Julia"))
close(fileln)

# read DA power decision
fileln = matopen(string("./pDA_",Zone,".mat"))
pDA_real = read(fileln, string("pS_DA"))
pDA_real = pDA_real[1:8760]
close(fileln)

inds = findall(i->(i<0), pDA_real);
neginds = getindex.(inds, 1);
inds = findall(i->(i>=0), pDA_real);
posinds = getindex.(inds, 1);

pDA_rdis = zeros(1,length(pDA_real));
pDA_rdis[posinds] .= pDA_real[posinds];
pDA_rdis = transpose(pDA_rdis);
# pDA_rdis = reshape(pDA_rdis,(24,365));

pDA_rchr = zeros(1,length(pDA_real));
pDA_rchr[neginds] .= -pDA_real[neginds];
pDA_rchr = transpose(pDA_rchr);
# pDA_rchr = reshape(pDA_rchr,(24,365));

RTP = reshape(RTP,(8760,1));
# RTP = RTP[1:24]
DAP = reshape(DAP,(8760,1));
# DAP = DAP[1:24];

In [34]:
pDA_real

8760-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.05666666666666671
  0.0
 -0.4988888888888888
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [32]:
## battery setting

E = 1;  # storage energy capacity in MWh
# simulation setting
T = 8760; # total time steps
P = .5; # power rating MW
eta = .9; # efficiency
e0 = .5 * E;
ef = e0;
MC = 10; # marginal discharge cost
M = 1

# initialize optimization model
model = Model(Gurobi.Optimizer)
set_silent(model) # no outputs

# @variable(model, d_da[1:T], lower_bound = 0) # discharge power in DA
@variable(model, d_rt[1:T], lower_bound = 0) # discharge power in RT
# @variable(model, c_da[1:T], lower_bound = 0) # charge power in DA
@variable(model, c_rt[1:T], lower_bound = 0) # charge power in RT
@variable(model, diff[1:T]) # difference between RT and DA power decision

@variable(model, e[1:T], lower_bound = 0) # total battery energy
@variable(model, R) # market revenue
@variable(model, C) # total degradation cost
@variable(model, u_rt[1:T], Bin) # 1 if charge/buy in RTM

@constraint(model, DiffPower[t=1:T], diff[t] - d_rt[t] + c_rt[t] == -pDA_real[t] ) # actual dis/charge power 

# arbitrage revenue
@constraint(model, ArbRev, R == M*sum(DAP.* pDA_real .+ RTP.* diff ) )
# @constraint(model, ArbRev, M*sum(RTP.* diff) - R == 0 )

# total degradation cost
@constraint(model, DegCost, M*sum(d_rt * MC) - C == 0 )
# @constraint(model, DegCost, C == M*sum(diff .* MC))

# initial SoC evolution
@constraint(model, SoCInit, e[1] - e0 == M*(c_rt[1]*eta - d_rt[1]/eta) )
# rest SoC evolution
@constraint(model, SoCCont[t = 2:T], e[t] - e[t-1] == M*(c_rt[t]*eta - d_rt[t]/eta) )

# final energy level
# @constraint(model, Enelast, e[T] >= ef )
@constraint(model, Enelast[t = 24:24:T], e[t] >= ef )

# charging / discharging non-conflict condition
@constraint(model, ChRatingTot[t=1:T], c_rt[t] <= P*u_rt[t])
@constraint(model, DchRatingTot[t=1:T], d_rt[t] <= P*(1-u_rt[t]))

# max energy level
@constraint(model, SoCMax[t=1:T], e[t] <= E )

# maximize revenue plus degradation value
@objective(model, Max, R-C);
# @objective(model, Max, R);
# print(model);
# ArbRev


Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


In [33]:
optimize!(model)
termination_status(model)

# R_s = zeros(T, 1)
# P_s = zeros(T, 1)
# C_s = zeros(T, 1)
# soc = zeros(T, 1)

global R_s = value(R)
global C_s = value(C)
global P_s = objective_value(model)


@printf("Cummulative Rev %d, Cummulative Profit %d, Cummulative Cost %d, OptStatus: %s \n", R_s, P_s, C_s, termination_status(model))

SoC = vec(value.(e))
RTP = vec(RTP)
DAP = vec(DAP)
DA_Charge = vec(pDA_rchr)
RT_Charge = vec(value.(c_rt))
DA_Discharge = vec(pDA_rdis)
RT_Discharge = vec(value.(d_rt))

df = DataFrame(DAP = DAP, RTP = RTP, DA_Charge = DA_Charge, RT_Charge = RT_Charge, DA_Discharge = DA_Discharge, RT_Discharge = RT_Discharge, SoC = SoC)
# df = DataFrame(RTP = RTT, DAP = DAP, DA_Charge = DAcharge, RT_Charge = RTcharge, DA_Discharge = DAdischarge, RT_Discharge = RTdischarge, Charge = charge, Discharge = discharge, Pactual = pactual, SoC = SoC)
CSV.write("dispatch_0911_1year.csv", df);

Cummulative Rev 20013, Cummulative Profit 15785, Cummulative Cost 4228, OptStatus: OPTIMAL 
